# [AlexNet]

*KU LeeDongGyu*

### Contents

1. Almost Original Alexnet
2. My Alexnet & Data Import
```
1) Data Import
2) Data Augmentation
3) Size = 64,
4) Size = 48,
```
3. Size = 48, No Early Stopping
```
1) Epoch = 50
2) Epoch = 100
```

### Install Packages

In [1]:
pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 3.9MB 38.0MB/s 
     |████████████████████████████████| 450kB 45.4MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


### Module

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/Colab Notebooks/project

/content/drive/My Drive/Colab Notebooks/project


In [0]:
import sys

In [5]:
# 모듈로 받을 경로 확인
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [0]:
# 내 노트북이 아닌, 전산실 컴퓨터의 colab에서 돌렸으므로, 다시돌리려면 경로 수정할것!
sys.path.append("/content/drive/My Drive/Colab Notebooks/project")

In [0]:
from lrn import LRN #만든 모듈, class
from f1score import macro_f1score, weighted_f1score
from pool_helper import PoolHelper

In [0]:
import pandas as pd
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten , ZeroPadding2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/project'

In [10]:
tf.__version__

'2.1.0'

In [ ]:
tf.test.gpu_device_name()

## 1. Almost Original Alexnet

In [0]:
# Alexnet을 최대한 논문에 가깝게 맞춰 모형작성.
# 또한, Data Augmentation은 컴퓨터 성능의 한계로 하지 않음.
# 출처 : https://github.com/eweill/keras-deepcv/blob/master/models/classification/alexnet.py

def Alexnet(img_shape=(227, 227, 3), n_classes=10, l2_reg=0.,weights=None):

    # Initialize model
    alexnet = Sequential()

    # Layer 1
    alexnet.add(Conv2D(96, (11, 11), input_shape=img_shape, strides=4,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer1_LRN'))
    alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

    # Layer 2
    alexnet.add(Conv2D(256, (5, 5), padding='same',strides=1,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer2_LRN'))
    alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))
    
    # Layer 3
    alexnet.add(Conv2D(384, (3, 3), padding='same', strides=1,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer3_LRN'))
    
    # Layer 4
    alexnet.add(Conv2D(384, (3, 3), padding='same'))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer4_LRN'))

    # Layer 5
    alexnet.add(Conv2D(256, (3, 3), padding='same'))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer5_LRN'))
    alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

    # Layer 6
    alexnet.add(Flatten())
    alexnet.add(Dense(4096,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 7
    alexnet.add(Dense(4096,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 8
    alexnet.add(Dense(n_classes))
    alexnet.add(Activation('softmax'))

    if weights is not None:
        alexnet.load_weights(weights)

    return alexnet

In [0]:
model = Alexnet(img_shape=(227, 227, 3), n_classes=7, l2_reg=0.,weights=None)

In [0]:
#모수가 데이터에 비해 굉장히 많지만, 일단 진행
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
layer1_LRN (LRN)             (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 256)       0         
_________________________________________________________________
layer2_LRN (LRN)             (None, 27, 27, 256)       0

Too many parameters compared to the data. 

## 2. My Alexnet

### 1) Data Import

In [0]:
# Alexnet을 개조
# Data Augmentation은 컴퓨터 성능의 한계로 못하기 때문에 변형함.

# 주의 !!!!기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!!!

# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)

In [0]:
# data handling
x_train = np.array(x_train).reshape([-1,48,48,3]) 
x_valid = np.array(x_valid).reshape([-1,48,48,3]) 
x_test = np.array(x_test).reshape([-1,48,48,3]) 

y_train=to_categorical(y_train) # one hot encoding
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [0]:
# data handling
size = 64 #적당한 크기로 잡음.
x_train = np.array(x_train).reshape([-1,48,48,3])

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,3],dtype="float32")

for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [0]:
x_train = x_train / 255 

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,3])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,3],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [0]:
 x_valid = x_valid / 255

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,3])
x_test_zoom = np.zeros([x_test.shape[0],size,size,3],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [0]:
x_test = x_test / 255

### 2) Data Augmentation

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=[0.9,1.0])

In [0]:
datagen_val = ImageDataGenerator()

In [0]:
xy_valid_zoom_gen = datagen_val.flow(x_valid_zoom,y_valid,batch_size=128)
xy_valid_gen = datagen_val.flow(x_valid,y_valid,batch_size=128)

### 3) Size = 64

In [0]:
# 다음의 절차로 모형을 개조한다.

# 1. 227의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. 다음과 같이 모형을 재구성한다.

# convolution layer
# 입력 : 64x64x1
# 첫번째 층 : 5x5 필터 24장, strides = 2 -> maxpooling 3x3 , stirdes = 2   ===> 14 x 14 x 24 feature map
# 두번째 층 : 3x3 필터 64장, strides = 1, padding = "same"                 ===> 14 x 14 x 64 feature map
# 세번째 층 : 3x3 필터 96장, strides = 1                                   ===> 12 x 12 x 96 feature map
# 네번째 층 : 3x3 필터 96장, strides = 1, padding = "same"                 ===> 12 x 12 x 96 feature map
# 다섯째 층 : 3x3 필터 64장, strides = 1 -> maxpooling 3x3 , strides = 2   ===> 4 x 4 x 64 feature map

# fc layer
# 여섯째 층 : 노드 256개, dropout = 0.5
# 일곱째 층 : 노드 256개, dropout = 0.5
# 여덟째 층 : 노드 7개



def Alexnet(img_shape=(64, 64, 3), n_classes=7, l2_reg=0.,
	weights=None):

	# Initialize model
	alexnet = Sequential()

	# Layer 1
	alexnet.add(Conv2D(24, (5, 5), input_shape=img_shape, strides=2,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer1_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 2
	alexnet.add(Conv2D(64, (3, 3), padding='same',strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer2_LRN'))

	# Layer 3
	alexnet.add(Conv2D(96, (3, 3),  strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer3_LRN'))

	# Layer 4
	alexnet.add(Conv2D(96, (3, 3), padding='same'))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer4_LRN'))

	# Layer 5
	alexnet.add(Conv2D(64, (3, 3)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer5_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 6
	alexnet.add(Flatten())
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))

	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 7
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 8
	alexnet.add(Dense(n_classes))
	alexnet.add(Activation('softmax'))

	if weights is not None:
		alexnet.load_weights(weights)

	return alexnet

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(64, 64, 3), n_classes=7, l2_reg=0.,weights=None)

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 30, 30, 24)        1824      
_________________________________________________________________
activation_8 (Activation)    (None, 30, 30, 24)        0         
_________________________________________________________________
layer1_LRN (LRN)             (None, 30, 30, 24)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 24)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 64)        13888     
_________________________________________________________________
activation_9 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
layer2_LRN (LRN)             (None, 14, 14, 64)       

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score',patience=3,verbose=1,mode='max')

In [0]:
# early stopping 
model.fit(datagen.flow(x_train_zoom,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_zoom_gen, validation_steps=len(x_valid_zoom)/128, epochs=100, callbacks=[early_stopping]) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 40s 180ms/step - loss: 1.8275 - accuracy: 0.2445 - macro_f1score: 1.1318e-04 - weighted_f1score: 2.8294e-05 - val_loss: 1.8759 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
225/224 [==============================] - 33s 145ms/step - loss: 1.8103 - accuracy: 0.2521 - macro_f1score: 1.8599e-04 - weighted_f1score: 2.7407e-05 - val_loss: 1.8533 - val_accuracy: 0.2513 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
225/224 [==============================] - 33s 145ms/step - loss: 1.7846 - accuracy: 0.2574 - macro_f1score: 0.0011 - weighted_f1score: 1.2574e-04 - val_loss: 1.7867 - val_accuracy: 0.2937 - val_macro_f1score: 0.0058 - val_weighted_f1score: 6.8769e-04
Epoch 4/100
225/224 [==============================] - 33s 146ms/step - loss: 1.7388

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 1s 197us/sample - loss: 1.2379 - accuracy: 0.5309 - macro_f1score: 0.2963 - weighted_f1score: 0.0523

Accuracy: 0.5309, Macro F1 Score: 0.2963, Weighted F1 Score: 0.0523


### 4) Size = 48

In [0]:
# 다음의 절차로 모형을 개조한다.

# 1. 227의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. 다음과 같이 모형을 재구성한다.

# convolution layer
# 입력 : 64x64x1
# 첫번째 층 : 5x5 필터 24장, strides = 2 -> maxpooling 3x3 , stirdes = 2   ===> 10 x 10 x 24 feature map
# 두번째 층 : 3x3 필터 64장, strides = 1, padding = "same"                 ===> 14 x 14 x 64 feature map
# 세번째 층 : 3x3 필터 96장, strides = 1                                   ===> 12 x 12 x 96 feature map
# 네번째 층 : 3x3 필터 96장, strides = 1, padding = "same"                 ===> 12 x 12 x 96 feature map
# 다섯째 층 : 3x3 필터 64장, strides = 1 -> maxpooling 3x3 , strides = 2   ===> 4 x 4 x 64 feature map

# fc layer
# 여섯째 층 : 노드 256개, dropout = 0.5
# 일곱째 층 : 노드 256개, dropout = 0.5
# 여덟째 층 : 노드 7개



def Alexnet(img_shape=(48, 48, 3), n_classes=7, l2_reg=0., weights=None):

	# Initialize model
	alexnet = Sequential()


	# Layer 1
	alexnet.add(Conv2D(24, (5, 5), input_shape=img_shape, strides=2,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer1_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 2
	alexnet.add(Conv2D(64, (3, 3), padding='same',strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer2_LRN'))

	# Layer 3
	alexnet.add(Conv2D(96, (3, 3),  strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer3_LRN'))

	# Layer 4
	alexnet.add(Conv2D(96, (3, 3), padding='same'))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer4_LRN'))

	# Layer 5
	alexnet.add(Conv2D(64, (3, 3)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer5_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 6
	alexnet.add(Flatten())
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))

	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 7
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 8
	alexnet.add(Dense(n_classes))
	alexnet.add(Activation('softmax'))

	if weights is not None:
		alexnet.load_weights(weights)

	return alexnet

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(48, 48, 3), n_classes=7, l2_reg=0.,weights=None)

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 22, 22, 24)        1824      
_________________________________________________________________
activation_16 (Activation)   (None, 22, 22, 24)        0         
_________________________________________________________________
layer1_LRN (LRN)             (None, 22, 22, 24)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 24)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 10, 10, 64)        13888     
_________________________________________________________________
activation_17 (Activation)   (None, 10, 10, 64)        0         
_________________________________________________________________
layer2_LRN (LRN)             (None, 10, 10, 64)       

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score',patience=3,verbose=1,mode='max')

In [0]:
# early stopping 
model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128, epochs=100, callbacks=[early_stopping]) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 29s 131ms/step - loss: 1.8256 - accuracy: 0.2454 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8703 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
225/224 [==============================] - 26s 115ms/step - loss: 1.8044 - accuracy: 0.2518 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8436 - val_accuracy: 0.2575 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
225/224 [==============================] - 26s 117ms/step - loss: 1.7642 - accuracy: 0.2777 - macro_f1score: 0.0115 - weighted_f1score: 0.0014 - val_loss: 1.7323 - val_accuracy: 0.3218 - val_macro_f1score: 0.0243 - val_weighted_f1score: 0.0028
Epoch 4/100
225/224 [==============================] - 26s 117ms/step - loss: 1.6617 - accur

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 1s 190us/sample - loss: 1.2257 - accuracy: 0.5284 - macro_f1score: 0.3075 - weighted_f1score: 0.0547

Accuracy: 0.5284, Macro F1 Score: 0.3075, Weighted F1 Score: 0.0547


## 3. Size = 48 , No Early Stopping
---

### 1) Epoch = 50

In [0]:
### early stopping 없이 ###

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(48, 48, 3), n_classes=7, l2_reg=0.,weights=None)

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:

model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128 ,epochs=50) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/50
225/224 [==============================] - 29s 131ms/step - loss: 1.8271 - accuracy: 0.2467 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8755 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/50
225/224 [==============================] - 27s 120ms/step - loss: 1.8154 - accuracy: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8642 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/50
225/224 [==============================] - 27s 121ms/step - loss: 1.8047 - accuracy: 0.2512 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8411 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/50
225/224 [==============================] - 27s 121ms/step - loss: 1.

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 1s 176us/sample - loss: 1.2387 - accuracy: 0.5215 - macro_f1score: 0.3033 - weighted_f1score: 0.0537

Accuracy: 0.5215, Macro F1 Score: 0.3033, Weighted F1 Score: 0.0537


### 2) Epoch = 100

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(48, 48, 3), n_classes=7, l2_reg=0.,weights=None)

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [30]:
model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128 ,epochs=100) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 36s 160ms/step - loss: 1.8289 - accuracy: 0.2445 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8782 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
225/224 [==============================] - 28s 126ms/step - loss: 1.8156 - accuracy: 0.2514 - macro_f1score: 4.2328e-05 - weighted_f1score: 9.5899e-06 - val_loss: 1.8796 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
225/224 [==============================] - 28s 126ms/step - loss: 1.8016 - accuracy: 0.2550 - macro_f1score: 0.0014 - weighted_f1score: 1.7645e-04 - val_loss: 1.7905 - val_accuracy: 0.2937 - val_macro_f1score: 0.0383 - val_weighted_f1score: 0.0047
Epoch 4/100
225/224 [==============================] - 29s 128ms/step - loss: 1.7100 - a

In [31]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 1s 263us/sample - loss: 1.1154 - accuracy: 0.5753 - macro_f1score: 0.4176 - weighted_f1score: 0.0672

Accuracy: 0.5753, Macro F1 Score: 0.4176, Weighted F1 Score: 0.0672
